In [1]:
# mac475의 ipython 표준 style을 적용함
from IPython.core.display import HTML
styles = open("../styles/custom.css", "r").read()
HTML( styles )

    0. Dataset 읽기

    1. Feature 추출 function

    2. Label Encoding function

    3. Label Encoding용 Dataset
    
        3.1 train/ test dataset merge

    4. Modeling

        4.1 Merged features중, modeling시 제거할 feature 관리

        4.2 Train : 연/ 월/ 일 Feature 추출

        4.3 Train : Tube와 merge통한 Feature 확장

        4.4 Train : Merged dataset Label Encoding 수행

        4.5 Train : 불필요 Feature 제거

        4.6 Test : Dataset 읽기

        4.7 Test : 연/ 월/ 일 Feature 추출

        4.8 Test : Tube와 merge통한 Feature 확장

        4.9 Test : Merged dataset Label Encoding 수행

        4.10 Test : 불필요 Featrue 제거

#0. Dataset 읽기

In [2]:
import pandas as pd
import numpy as np

df_train = pd.read_csv( './dataset/train_set.csv' )    # data를 읽어들인다.
df_test = pd.read_csv( './dataset/test_set.csv' )    # data를 읽어들인다.

#1. Feature 추출 function
* quote_date 활용하여 year, month, day 확보

In [3]:
import datetime

def extract_year_month_day_from_quote_date( p_df ) :
    p_df[ 'quote_date' ] = pd.to_datetime( p_df[ 'quote_date' ] )    # string을 datetime으로 형변환
    p_df[ 'year' ] = p_df[ 'quote_date' ].dt.year    # 연도
    p_df[ 'month' ] = p_df[ 'quote_date' ].dt.month    # 월
    p_df[ 'day' ] = p_df[ 'quote_date' ].dt.day    # 일

    return p_df

#2. Label Encoding function
* 현재, train/ test dataset과 tube dataset과 merge하는 case까지 반영하여 encoder 관리

In [4]:
from sklearn import preprocessing

# le_tube_assembly_id = preprocessing.LabelEncoder()
le_supplier = preprocessing.LabelEncoder()
le_bracket_pricing = preprocessing.LabelEncoder()

le_material_id = preprocessing.LabelEncoder()
le_end_a_1x = preprocessing.LabelEncoder()
le_end_a_2x = preprocessing.LabelEncoder()
le_end_x_1x = preprocessing.LabelEncoder()
le_end_x_2x = preprocessing.LabelEncoder()
le_end_a = preprocessing.LabelEncoder()
le_end_x = preprocessing.LabelEncoder()

def executeLabelEncoding( p_df, is_init ) :
    if is_init == True :    # training dataset인 경우, label encoder 생성 및 fitting 수행
        p_df[ 'bracket_pricing' ] = le_bracket_pricing.fit_transform( p_df[ 'bracket_pricing' ] )

        p_df[ 'supplier' ] = le_supplier.fit_transform( p_df[ 'supplier' ] )
        p_df[ 'material_id' ] = le_material_id.fit_transform( p_df[ 'material_id' ] )
        
        p_df[ 'end_a_1x' ] = le_end_a_1x.fit_transform( p_df[ 'end_a_1x' ] )
        p_df[ 'end_a_2x' ] = le_end_a_2x.fit_transform( p_df[ 'end_a_2x' ] )
        p_df[ 'end_x_1x' ] = le_end_x_1x.fit_transform( p_df[ 'end_x_1x' ] )
        p_df[ 'end_x_2x' ] = le_end_x_2x.fit_transform( p_df[ 'end_x_2x' ] )
        p_df[ 'end_a' ] = le_end_a.fit_transform( p_df[ 'end_a' ] )
        p_df[ 'end_x' ] = le_end_x.fit_transform( p_df[ 'end_x' ] )
    else :
        p_df[ 'bracket_pricing' ] = le_bracket_pricing.transform( p_df[ 'bracket_pricing' ] )

        p_df[ 'supplier' ] = le_supplier.transform( p_df[ 'supplier' ] )
        p_df[ 'material_id' ] = le_material_id.transform( p_df[ 'material_id' ] )
        
        p_df[ 'end_a_1x' ] = le_end_a_1x.transform( p_df[ 'end_a_1x' ] )
        p_df[ 'end_a_2x' ] = le_end_a_2x.transform( p_df[ 'end_a_2x' ] )
        p_df[ 'end_x_1x' ] = le_end_x_1x.transform( p_df[ 'end_x_1x' ] )
        p_df[ 'end_x_2x' ] = le_end_x_2x.transform( p_df[ 'end_x_2x' ] )
        p_df[ 'end_a' ] = le_end_a.transform( p_df[ 'end_a' ] )
        p_df[ 'end_x' ] = le_end_x.transform( p_df[ 'end_x' ] )
    
    return p_df

.

#3. Label Encoding용 Dataset
* categorical data를 모두 포함해야 하므로, tube join후 기반으로 label encoder를 fitting한다
* label encoding의 대상은 bracket_pricing, supplier, material_id, end_a_1x 시리즈, end_a 시리즈 feature이다

##3.1 train/ test dataset merge

In [5]:
df_train[ 'id' ] = '99999'    # test와 join위해 feature 추가 : 99999는 train dataset이다

In [6]:
df_merged = df_train.append( df_test )    # train과 test df를 merge한다.

In [7]:
df_merged.tail( 3 )

,annual_usage,bracket_pricing,cost,id,min_order_quantity,quantity,quote_date,supplier,tube_assembly_id
30232,883,No,NaN,30233,85,85,2013-09-10,S-0041,TA-21194
30233,28,No,NaN,30234,1,1,2003-09-16,S-0026,TA-21198
30234,28,No,NaN,30235,1,1,2003-09-16,S-0026,TA-21199


In [8]:
df_merged.count()

annual_usage          60448
bracket_pricing       60448
cost                  30213
id                    60448
min_order_quantity    60448
quantity              60448
quote_date            60448
supplier              60448
tube_assembly_id      60448
dtype: int64

##3.1 tube dataset merge 및 label encoding

In [9]:
df_tube = pd.read_csv( './dataset/02.ml.verified.dataset/tube.material_id.verified.csv' )

In [10]:
df_merged = df_merged.merge( df_tube, how = 'inner', on = 'tube_assembly_id' )

In [11]:
df_merged.count()

annual_usage          60448
bracket_pricing       60448
cost                  30213
id                    60448
min_order_quantity    60448
quantity              60448
quote_date            60448
supplier              60448
tube_assembly_id      60448
material_id           60448
diameter              60448
wall                  60448
length                60448
num_bends             60448
bend_radius           60448
end_a_1x              60448
end_a_2x              60448
end_x_1x              60448
end_x_2x              60448
end_a                 60448
end_x                 60448
num_boss              60448
num_bracket           60448
other                 60448
dtype: int64

In [12]:
df_merged.columns    # merged feature를 확인

Index(['annual_usage', 'bracket_pricing', 'cost', 'id', 'min_order_quantity', 'quantity', 'quote_date', 'supplier', 'tube_assembly_id', 'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other'], dtype='object')

In [13]:
df_merged = executeLabelEncoding( df_merged, is_init = True )    # label encoding을 수행한다

In [14]:
df_train.drop( 'id', axis = 1, inplace = True )    # 일단, 필요없는 것들 제거 : df_train을 원상 복구해둠
df_test = ''    # 일단, 필요없는 것들 제거
df_merged = ''    # 일단, 필요없는 것들 제거

#4. Modeling

##4.1 Merged features중, modeling시 제거할 feature 관리

In [15]:
def executeFeatureRemoval( p_df ) :
    # 제거하고자 하는 feature list
#     list_for_remove = [    # 39.8% 정확도 feature list
#                         'tube_assembly_id',
# #                         'supplier',

#                         'quote_date',
#                         'annual_usage',
#                         'min_order_quantity',
        
#     #                     'bracket_pricing',
#     #                     'quantity',
        
#                         'year',
#                         'month',
#                         'day',

# #                         'material_id',
# #                         'diameter',
# #                         'wall',
# #                         'length',
    
# #                         'num_bends',
# #                         'bend_radius',
    
# #                         'end_a_1x',
# #                         'end_a_2x',
# #                         'end_x_1x',
# #                         'end_x_2x',
# #                         'end_a',
# #                         'end_x',
    
# #                         'num_boss',
# #                         'num_bracket',
# #                         'other',
#                       ]
    
#     list_for_remove = [    # 44.2% 정확도 feature list
#                         'tube_assembly_id',
# #                         'supplier',

#                         'quote_date',
#                         'annual_usage',
#                         'min_order_quantity',
        
#     #                     'bracket_pricing',
#     #                     'quantity',
        
# #                         'year',
# #                         'month',
#                         'day',

# #                         'material_id',
# #                         'diameter',
# #                         'wall',
# #                         'length',
    
# #                         'num_bends',
# #                         'bend_radius',
    
# #                         'end_a_1x',
# #                         'end_a_2x',
# #                         'end_x_1x',
# #                         'end_x_2x',
# #                         'end_a',
# #                         'end_x',
    
# #                         'num_boss',
# #                         'num_bracket',
# #                         'other',
#                       ]

#     list_for_remove = [    # 59.2% 정확도 feature list
#                         'tube_assembly_id',
# #                         'supplier',

#                         'quote_date',
# #                         'annual_usage',
# #                         'min_order_quantity',
        
#     #                     'bracket_pricing',
#     #                     'quantity',
        
# #                         'year',
# #                         'month',
#                         'day',

# #                         'material_id',
# #                         'diameter',
# #                         'wall',
# #                         'length',
    
# #                         'num_bends',
# #                         'bend_radius',
    
# #                         'end_a_1x',
# #                         'end_a_2x',
# #                         'end_x_1x',
# #                         'end_x_2x',
# #                         'end_a',
# #                         'end_x',
    
# #                         'num_boss',
# #                         'num_bracket',
# #                         'other',
#                       ]   
    
    list_for_remove = [    # 61.0% 정확도 feature list
                        'tube_assembly_id',
#                         'supplier',

                        'quote_date',
#                         'annual_usage',
#                         'min_order_quantity',
        
    #                     'bracket_pricing',
    #                     'quantity',
        
#                         'year',
#                         'month',
#                         'day',

#                         'material_id',
#                         'diameter',
#                         'wall',
#                         'length',
    
#                         'num_bends',
#                         'bend_radius',
    
#                         'end_a_1x',
#                         'end_a_2x',
#                         'end_x_1x',
#                         'end_x_2x',
#                         'end_a',
#                         'end_x',
    
#                         'num_boss',
#                         'num_bracket',
#                         'other',
                      ]       

    return p_df.drop( list_for_remove, axis = 1, inplace = False )

##4.2 Train : 연/ 월/ 일 Feature 추출

In [16]:
df_train = extract_year_month_day_from_quote_date( df_train )    # feature 처리를 수행

##4.3 Train : Tube와 merge통한 Feature 확장

In [17]:
df_train_merged = df_train.merge( df_tube, how = 'inner', on = 'tube_assembly_id' )

##4.4 Train : Merged dataset Label Encoding 수행

In [18]:
df_train_merged = executeLabelEncoding( df_train_merged, is_init = False )    # label encoding을 수행한다

##4.5 Train : 불필요 Feature 제거

In [19]:
df_train_merged = executeFeatureRemoval( df_train_merged )    # feature removal 처리를 수행
df_train_merged.head( 3 )

,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,cost,year,month,day,material_id,...,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,41,0,0,1,1,21.905933,2013,7,7,1,...,19.05,0,0,0,0,7,7,0,0,0
1,41,0,0,1,2,12.341214,2013,7,7,1,...,19.05,0,0,0,0,7,7,0,0,0
2,41,0,0,1,5,6.601826,2013,7,7,1,...,19.05,0,0,0,0,7,7,0,0,0


In [20]:
df_train_merged.shape

(30213, 24)

.

##4.6 Test : Dataset 읽기

In [21]:
df_test = pd.read_csv( './dataset/test_set.csv' )    # data를 읽어들인다.

##4.7 Test : 연/ 월/ 일 Feature 추출

In [22]:
df_test = extract_year_month_day_from_quote_date( df_test )    # feature 처리를 수행

df_result = pd.DataFrame( df_test[ 'id' ], columns = ['id'] )    # 결과용 dataframe을 생성
df_test.drop( 'id', axis = 1, inplace = True )    # id feature는 일단 제거

##4.8 Test : Tube와 merge통한 Feature 확장

In [23]:
df_test_merged = df_test.merge( df_tube, how = 'inner', on = 'tube_assembly_id' )
# df_test_merged = df_test.merge( df_tube, how = 'outer', on = 'tube_assembly_id' )

In [24]:
df_test_merged.head( 3 )

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,day,...,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,TA-00001,S-0066,2013-06-23,0,0,Yes,1,2013,6,23,...,38.1,N,N,N,N,EF-003,EF-003,0,0,0
1,TA-00001,S-0066,2013-06-23,0,0,Yes,2,2013,6,23,...,38.1,N,N,N,N,EF-003,EF-003,0,0,0
2,TA-00001,S-0066,2013-06-23,0,0,Yes,5,2013,6,23,...,38.1,N,N,N,N,EF-003,EF-003,0,0,0


##4.9 Test : Merged dataset Label Encoding 수행

In [25]:
df_test_merged = executeLabelEncoding( df_test_merged, is_init = False )    # label encoding을 수행한다

In [26]:
df_test_merged.head( 3 )

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,day,...,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,TA-00001,41,2013-06-23,0,0,1,1,2013,6,23,...,38.1,0,0,0,0,2,2,0,0,0
1,TA-00001,41,2013-06-23,0,0,1,2,2013,6,23,...,38.1,0,0,0,0,2,2,0,0,0
2,TA-00001,41,2013-06-23,0,0,1,5,2013,6,23,...,38.1,0,0,0,0,2,2,0,0,0


##4.10 Test : 불필요 Featrue 제거

In [27]:
df_test_merged = executeFeatureRemoval( df_test_merged )    # feature removal 처리를 수행
df_test_merged.head( 3 )

,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,day,material_id,diameter,...,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,41,0,0,1,1,2013,6,23,9,12.7,...,38.1,0,0,0,0,2,2,0,0,0
1,41,0,0,1,2,2013,6,23,9,12.7,...,38.1,0,0,0,0,2,2,0,0,0
2,41,0,0,1,5,2013,6,23,9,12.7,...,38.1,0,0,0,0,2,2,0,0,0


In [28]:
df_train_merged.shape, df_test_merged.shape

((30213, 24), (30235, 23))

##4.11 Prediction Modeling

In [29]:
df_train_merged.head( 3 )

,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,cost,year,month,day,material_id,...,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,41,0,0,1,1,21.905933,2013,7,7,1,...,19.05,0,0,0,0,7,7,0,0,0
1,41,0,0,1,2,12.341214,2013,7,7,1,...,19.05,0,0,0,0,7,7,0,0,0
2,41,0,0,1,5,6.601826,2013,7,7,1,...,19.05,0,0,0,0,7,7,0,0,0


In [30]:
X = df_train_merged.drop( 'cost', axis = 1, inplace = False )    # X를 확보
y = np.log1p( df_train_merged[ 'cost' ] )   # y를 확보

df_train = ''
df_train_merged = ''

In [31]:
X.head( 3 )

,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,year,month,day,material_id,diameter,...,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,41,0,0,1,1,2013,7,7,1,6.35,...,19.05,0,0,0,0,7,7,0,0,0
1,41,0,0,1,2,2013,7,7,1,6.35,...,19.05,0,0,0,0,7,7,0,0,0
2,41,0,0,1,5,2013,7,7,1,6.35,...,19.05,0,0,0,0,7,7,0,0,0


In [32]:
y.head( 3 )

0    3.131396
1    2.590858
2    2.028389
Name: cost, dtype: float64

In [33]:
X.shape, y.shape

((30213, 23), (30213,))

In [34]:
cv_cnt = 10    # cv : cross validation 횟수
n_jobs_cnt = 6

In [35]:
from sklearn.linear_model import SGDRegressor

from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn import grid_search

In [36]:
model_list = [
#                Lasso(),
#                ElasticNet(),
#                KernelRidge(),
#                SVR( kernel = 'linear' ),
#                SVR( kernel = 'rbf' ),
               RandomForestRegressor( max_depth = 50, n_estimators = 50, n_jobs = n_jobs_cnt ),
#                LinearRegression(),    
             ]

In [37]:
X.shape, y.shape, df_test_merged.shape

((30213, 23), (30213,), (30235, 23))

In [38]:
%%time
import time
for model in model_list :
    params = {
#                'max_depth' : (10,30,50),    # RandomForest
#                'n_estimators' : (10,20,50,100),    # RandomForest
             }

    gs = grid_search.GridSearchCV( model,
                                   param_grid = params, n_jobs = n_jobs_cnt, cv = cv_cnt, )
    gs.fit( X, y )
    
    
    print( 'model : ', str( model ).split( '(' )[0] )
    print( 'best_score : ', gs.best_score_ )
    
    print( '=================' )
    print( 'best model : ', gs.best_estimator_ )
    print( '=================' )
    
#     df_tmp = pd.DataFrame( X.columns.values, columns = [ 'features' ] )
#     df_tmp[ 'importance' ] = gs.best_estimator_.feature_importances_
#     print( df_tmp.sort( 'importance', ascending = False  ) )    
    
    
    y_pred = gs.best_estimator_.predict( df_test_merged )    # prediction 수행
    df_result[ 'cost' ] = np.expm1( y_pred )
    
#     now = time.strftime( '%Y%m%d%H%M%S' )    # 현재시각을 확보
#     model_name = str( model ).split( '(' )[0]    # 파일생성용
#     file_timestamp = now[2:4] + now[4:6] + now[6:8] + now[8:10] + now[10:12] + now[12:14]
#     accuracy = '{0:.1f}%'.format( gs.best_score_ * 100 )    # latitude에 대한 예측률 저장 : 파일명 활용용도임
#     df_result.to_csv( path_or_buf = './result.to.submit/' + file_timestamp + '.' + model_name +
#                       '.' + accuracy + '.result().csv', sep = ',', index = False )

model :  RandomForestRegressor
best_score :  0.82793108883
best model :  RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=6, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
Wall time: 15.9 s


In [39]:
df_result.head( 3 )

,id,cost
0,1,23.592323
1,2,13.659908
2,3,7.861669


In [40]:
df_result_desc = df_result.describe()    # 결과의 overview 확인
df_result_desc.drop( 'id', axis = 1, inplace = True )
df_result_desc.describe()

,cost
count,8.000000
mean,3888.610055
std,10649.287756
min,0.908510
25%,5.992248
50%,12.774651
75%,219.600336
max,30235.000000


In [41]:
X = ''    # X 초기화
y = ''
df_test = ''
df_result = ''
df_tmp = ''

#참고 code : sklearn RF
[LINK]('https://www.kaggle.com/pakozm/caterpillar-tube-pricing/sklearn-rf')

In [42]:
# """
# Beating the Benchmark 
# Caterpillar @ Kaggle

# __author__ : pakozm

# forked from Abhishek version

# """

# import pandas as pd
# import numpy as np
# import sklearn as sk
# from sklearn import ensemble, preprocessing
# from sklearn.cross_validation import train_test_split
# from sklearn import ensemble

# NUM_TREES=1000
# MAX_FEATS="sqrt"

# # load training and test datasets
# train = pd.read_csv('../input/train_set.csv', parse_dates=[2,])
# test = pd.read_csv('../input/test_set.csv', parse_dates=[3,])

# # create some new features
# train['year'] = train.quote_date.dt.year
# train['month'] = train.quote_date.dt.month
# train['dayofyear'] = train.quote_date.dt.dayofyear
# train['dayofweek'] = train.quote_date.dt.dayofweek
# train['day'] = train.quote_date.dt.day

# test['year'] = test.quote_date.dt.year
# test['month'] = test.quote_date.dt.month
# test['dayofyear'] = test.quote_date.dt.dayofyear
# test['dayofweek'] = test.quote_date.dt.dayofweek
# test['day'] = test.quote_date.dt.day

# # drop useless columns and create labels
# idx = test.id.values.astype(int)
# test = test.drop(['id', 'tube_assembly_id', 'quote_date'], axis = 1)
# labels = train.cost.values
# train = train.drop(['quote_date', 'cost', 'tube_assembly_id'], axis = 1)

# # convert data to numpy array
# train = np.array(train)
# test = np.array(test)

# # label encode the categorical variables
# for i in range(train.shape[1]):
#     if i in [0,3]:
#         lbl = preprocessing.LabelEncoder()
#         lbl.fit(list(train[:,i]) + list(test[:,i]))
#         train[:,i] = lbl.transform(train[:,i])
#         test[:,i] = lbl.transform(test[:,i])


# # object array to float
# train = train.astype(float)
# test = test.astype(float)

# # i like to train on log(1+x) for RMSLE ;) 
# # The choice is yours :)
# label_log = np.log1p(labels)

# # fit a random forest model
# def fit(train_feats, train_labels):
#     model = ensemble.RandomForestRegressor(n_estimators=NUM_TREES,
#                                            max_features=MAX_FEATS,
#                                            random_state=123,
#                                            verbose=100,
#                                            criterion="mse")
#     model.fit(train_feats, train_labels)
#     return model

# model = fit(train, label_log)
# train_p = model.predict(train)
# print ("# TR LOSS",np.sqrt(sk.metrics.mean_squared_error(train_p, label_log)))

# # get predictions from the model, convert them and dump them!
# test_p = np.expm1(model.predict(test))

# preds = pd.DataFrame({"cost": test_p}, index = idx)
# preds.to_csv('benchmark.csv', index=True, index_label="id")

# # Swipe right on tinder ;)
